In [1]:
from functions_simulator import *
import torch
from sbi.utils import BoxUniform
from sbi.inference import SNPE_A
import numpy as np

Create synthetic data to test the SBI method

In [ ]:
num_dims = 2 #s and eta
num_sims = 1000

eta_inf = 0.09
eta_sup = 2
s_inf = 0.04
s_sup = 0.2

nb_obj_max = 100

prior = BoxUniform(low=torch.tensor([eta_inf,s_inf]), high=torch.tensor([eta_sup,s_sup]))


def simulator_sbi(theta):
    """
    This function takes in argument the parameters theta and returns simulations of the models with these parameters 
    which can then be used to infer the parameters with the SBI package.

    Input :
    theta : torch tensor containing the parameters eta and s.

    Output : 
    torch tensor containing the network simulations with parameter theta.
    """
    num_sims = theta.shape[0] #number of simulations
    res = []
    if num_sims == 2:
        cond = True #becomes false if the learning task is achieved
        count = 0
        while cond:
            eta = theta.numpy()[0]
            s = theta.numpy()[1]
            nb_iter, times, accuracy = simulator_vectorized(eta, s, nb_obj_max=nb_obj_max)
            #print(nb_iter)
            if nb_iter < nb_obj_max + 1 :
                cond = False
            if count > 100:
                print('count too big')
                cond = False
            count +=1
        res.append(times)
        #print(count)
    else:
        for n in range(num_sims):
            cond = True #becomes false if the learning task is achieved
            count = 0
            while cond:
                eta = theta.numpy()[n,0]
                s = theta.numpy()[n,1]
                nb_iter, times, accuracy = simulator_vectorized(eta, s, nb_obj_max=nb_obj_max)
                if nb_iter < nb_obj_max + 1 :
                    cond = False
                if count > 100:
                    print('count too big')
                    cond = False
                count +=1
            res.append(times)
    res = np.array(res)
    return torch.FloatTensor(res)

#parameters used to produce the synthetic data
eta_0 = 1 
s_0 = 0.15
theta_0 = torch.tensor([eta_0,s_0])

x_0 = simulator_sbi(theta_0) #synthetic data 

Run the SBI method to get posteriors on the parameters eta and s

In [ ]:
num_rounds = 2

inference = SNPE_A(prior)
proposal = prior
for _ in range(num_rounds):
    theta = proposal.sample((num_sims,))
    x = simulator_sbi(theta)
    _ = inference.append_simulations(theta, x, proposal=proposal).train()
    posterior = inference.build_posterior().set_default_x(x_0)
    proposal = posterior

Save samples of the posteriors

In [8]:
torch.save(posterior, f'/Users/sophiejaffard/Desktop/Expé/saves_sbi/posterior_{eta_inf}_{eta_sup}_{s_inf}_{s_sup}_{eta_0}_{s_0}.pt')
posterior = torch.load(f'/Users/sophiejaffard/Desktop/Expé/saves_sbi/posterior_{eta_inf}_{eta_sup}_{s_inf}_{s_sup}_{eta_0}_{s_0}.pt')

sample = posterior.sample((5000,))
torch.save(sample, f'/Users/sophiejaffard/Desktop/Expé/saves_sbi/sample_{eta_inf}_{eta_sup}_{s_inf}_{s_sup}_{eta_0}_{s_0}.pt')